In [4]:
#!pip install geemap

import ee, geemap, pprint, folium
import geopandas as gpd
#ee.Authenticate()

def build_map(lat, lon, zoom, vizParams, image, name):
    map = geemap.Map(center = [lat, lon], zoom = zoom)
    map.addLayer(image, vizParams, name)
    return map
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

# Initialize the Earth Engine module.
ee.Initialize()

# Operations within GEE

Now that we have learned to reference raster objects in GEE and bring in vector objects (whether via geopandas or directly from a shapefile), we can start obtaining more data. 

## Part 1: Clip Raster to Vector

For demonstration purposes we will use the geopandas data to get started. To begin, we will select a country in central america, where the countries are relatively small. 



In [18]:
# Country boundaries within geopandas 
world_filepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(world_filepath)
df_single = world.loc[world.continent == 'Costa Rica']
df_single.plot()

In [33]:
# Convert to geemap object
# Create a GEE Feature
roi = geemap.geopandas_to_ee(df_single)

c:\Users\ozzyc\anaconda3\envs\geo\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [34]:
Map = geemap.Map()
Map.addLayer(roi, {}, "geopandas to ee example")
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [44]:
lat = 8.9; lon = -82.32
zoom = 8
image_collection_name = "LANDSAT/LC08/C01/T1_8DAY_NDVI"
date_start = '2017-01-01'
date_end = '2017-12-31'
name = 'Landsat'

image = (
    ee.ImageCollection(image_collection_name).mosaic()
)
# Note - we are building a mosaic of the data 
# and then clipping 
image = image.clip(roi)

palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301']

vizParams = {
    'min': 0, 
    'max': 0.1,
    'palette': palette, 
    'opacity': 0.6
}

map = build_map(lat, lon, zoom, vizParams, image, name)
map

Map(center=[8.9, -82.32], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [45]:
# Now we need to conduct zonal statistics 
output = 'stats.csv'
geemap.zonal_statistics(image, roi, output, statistics_type='MEAN', scale=30)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\ozzyc\OneDrive\03_Projects\03_GEE_Labs\03_GEE_DSPG\stats.csv
